# Training HANet 
This notebook is to show how to train HANet and visulize the grasping estimation. Our network is a FCN whose backbone is ResNet-101 and pre-trianed by [ImageNet](https://www.image-net.org/download.php).

Considering that some people are concerned about the invasion of privacy while capturing the photo, we offer two training method by inputting different datasets. 

The inputs of a general HANet are RGB and depth images.

The inputs of a privacy-protect HANet are only depth images, which can avoid seeing faces in the training photo.

## HANet(General Version)

## DataLoader and hyper parameters

Training HANet on HANet training datasets with training parameters below.

Batch Size : 5

Epoch : 50

Learning Rate :　0.001

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
from torch.utils.data import  DataLoader
from handover_grasping.model import HANet, HANet_depth
from handover_grasping.datavisualizer import handover_grasping_dataset

BATCH_SIZE = 5
EPOCH = 100
SAVE_EVERY = 25
DATA_PATH = '/home/arg/handover_grasping/data/HANet_training_datasets'

dataset = handover_grasping_dataset(DATA_PATH, color_type='png')
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 8)

## Initialize and show the structure of HANet

In [6]:
net = HANet(4)
net = net.cuda()
net

HANet(
  (net): FCN_model(
    (color_trunk): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=

## Set Loss function and optimizer

Using [Binary Cross-Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)

In [7]:
criterion = nn.BCEWithLogitsLoss().cuda()

optimizer = optim.Adam(net.parameters(), lr = 1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 25, gamma = 0.1)

## Start Training

Training 50 epoch and save weight every 25 epoch.

In [ ]:
if os.path.isdir(DATA_PATH + '/weight') == False:
    os.mkdir(DATA_PATH + '/weight')
    
loss_l = []
for epoch in range(EPOCH):
    loss_sum = 0.0
    ts = time.time()
    for i_batch, sampled_batched in enumerate(dataloader):
        print("\r[{:03.2f} %]".format(i_batch/float(len(dataloader))*100.0), end="\r")
        optimizer.zero_grad()
        color = sampled_batched['color'].cuda()
        depth = sampled_batched['depth'].cuda()
        label = sampled_batched['label'].permute(0,2,3,1).cuda().float()
        predict = net(color, depth)

        loss = criterion(predict, label)

        loss.backward()
        loss_sum += loss.detach().cpu().numpy()
        optimizer.step()
    scheduler.step()
    loss_l.append(loss_sum/len(dataloader))
    if (epoch+1)%SAVE_EVERY==0:
        torch.save(net.state_dict(), DATA_PATH + '/weight/HANet_{}_{}.pth' .format(epoch+1, round(loss_l[-1],3)))

    print("Epoch: {}| Loss: {}| Time elasped: {}".format(epoch+1, round(loss_l[-1],5), time.time()-ts))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Epoch: 1| Loss: 0.03081| Time elasped: 41.048171281814575
Epoch: 2| Loss: 0.02103| Time elasped: 40.738622426986694
Epoch: 3| Loss: 0.01946| Time elasped: 40.7121217250824
Epoch: 4| Loss: 0.01758| Time elasped: 40.868228912353516
Epoch: 5| Loss: 0.01607| Time elasped: 40.72752022743225
Epoch: 6| Loss: 0.01458| Time elasped: 40.84485340118408
Epoch: 7| Loss: 0.01389| Time elasped: 40.63869309425354
Epoch: 8| Loss: 0.01249| Time elasped: 40.737128496170044
Epoch: 9| Loss: 0.01133| Time elasped: 40.82390546798706
Epoch: 10| Loss: 0.00981| Time elasped: 41.09166359901428
Epoch: 11| Loss: 0.00926| Time elasped: 40.20421600341797
Epoch: 12| Loss: 0.0081| Time elasped: 38.627933502197266
Epoch: 13| Loss: 0.0078| Time elasped: 38.868866205215454
Epoch: 14| Loss: 0.00649| Time elasped: 38.26200485229492
Epoch: 15| Loss: 0.00647| Time elasped: 38.02544069290161
Epoch: 16| Loss: 0.00557| Time elasped: 38.75829553604126
Epoch: 17| Loss: 0.00569| Time elasped: 38.697975873947144
Epoch: 18| Loss: 0.

## HANet (Privacy-protect version)
The below code is aim to train a HANet for inputing depth images only.
The depth images will replicate to replace the input of RGB image.

## DataLoader and parameters

Training HANet on HANet training datasets with training parameters below.

Batch Size : 5

Epoch : 50

Learning Rate :　0.001

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
from torch.utils.data import  DataLoader
from handover_grasping.model import HANet, HANet_depth
from handover_grasping.datavisualizer import handover_grasping_dataset

BATCH_SIZE = 5
EPOCH = 50
SAVE_EVERY = 25
DATA_PATH = '/home/arg/handover_grasping/HANet_training_datasets'

dataset = handover_grasping_dataset(DATA_PATH, color_type='png')
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 8)

##  Initialize and show the structure of HANet

In [ ]:
net = HANet_depth(4)
net = net.cuda()

criterion = nn.BCEWithLogitsLoss().cuda()

optimizer = optim.Adam(net.parameters(), lr = 1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 25, gamma = 0.1)

## Start Training

Training 50 epoch and save weight every 25 epoch.

In [ ]:
if os.path.isdir(DATA_PATH + '/weight') == False:
    os.mkdir(DATA_PATH + '/weight')
    
loss_l = []
for epoch in range(EPOCH):
    loss_sum = 0.0
    ts = time.time()
    for i_batch, sampled_batched in enumerate(dataloader):
        print("\r[{:03.2f} %]".format(i_batch/float(len(dataloader))*100.0), end="\r")
        optimizer.zero_grad()
        depth = sampled_batched['depth'].cuda()
        label = sampled_batched['label'].permute(0,2,3,1).cuda().float()
        predict = net(depth)

        loss = criterion(predict, label)

        loss.backward()
        loss_sum += loss.detach().cpu().numpy()
        optimizer.step()
    scheduler.step()
    loss_l.append(loss_sum/len(dataloader))
    if (epoch+1)%SAVE_EVERY==0:
        torch.save(net.state_dict(), DATA_PATH + '/weight/HANet_depth_{}_{}.pth' .format(epoch+1, round(loss_l[-1],3)))

    print("Epoch: {}| Loss: {}| Time elasped: {}".format(epoch+1, round(loss_l[-1],5), time.time()-ts))